<a href="https://colab.research.google.com/github/AngelCastroRubio/ExaU2/blob/main/Etapa%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parte 1, correspondiente a la Importación de datos y módulos básicos, Visualización y Análisis Exploratorio del Conjunto de Datos y a las Estadísticas de Resumen y Coeficiente de Variación

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#----Visualización y Análisis Exploratorio del Conjunto de Datos----
# Carga del conjunto de datos
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# Visualización de las primeras filas del conjunto de datos
print(data.head())

# Verificación de tipos de datos
print(data.dtypes)

# Análisis de valores únicos
print(data.nunique())

#----Estadísticas de Resumen y Coeficiente de Variación----
# Estadísticas descriptivas
print(data.describe())

# Seleccionar solo columnas numéricas
data_numeric = data.select_dtypes(include=[np.number])

# Calcular la desviación estándar y el coeficiente de variación
std = data_numeric.std()
mean = data_numeric.mean()
cv = std / mean

# Mostrar los resultados
print("Desviacion estandar:")
print(std)
print("\nCoeficiente de variacion:")
print(cv)


Etapa 2: Preparación de los Datos. Se siguen los siguientes pasos: Eliminación de Variables Redundantes, Comprobación e Imputación de Valores Perdidos, Identificación y Limitación de Valores Atípicos, Codificación de Características, Importancia de las Características y Eliminación de la Multicolinealidad

In [ ]:
# Eliminamos la variable ID del DataSet
data.drop('id', axis=1, inplace=True)

# Realizamos la Comprobación e Imputación a BMI usando la mediana
median_bmi = data['bmi'].median()
data.loc[:, 'bmi'] = data['bmi'].fillna(median_bmi)

# Limitamos los valores atípicos de nuestra muestra con los percentiles
q_low = data['avg_glucose_level'].quantile(0.05)
q_hi = data['avg_glucose_level'].quantile(0.95)
data = data[(data['avg_glucose_level'] >= q_low) & (data['avg_glucose_level'] <= q_hi)]

q_low = data['bmi'].quantile(0.05)
q_hi = data['bmi'].quantile(0.95)
data = data[(data['bmi'] >= q_low) & (data['bmi'] <= q_hi)]

# Codificamos las caracteristicas actuales y las usamos numericamente
data_encoded = pd.get_dummies(data)
print(data_encoded.columns)

# Renombrar la columna 'stroke' a 'ictus'
data_encoded.rename(columns={'stroke': 'ictus'}, inplace=True)

# Eliminar la columna 'ictus' del conjunto de datos
X = data_encoded.drop('ictus', axis=1)

# Importancia de las características
from xgboost import XGBClassifier
X = data_encoded.drop('ictus', axis=1)
y = data_encoded['ictus']
model = XGBClassifier()
model.fit(X, y)
feature_importances = model.feature_importances_
print(X.columns)

# Eliminación de la multicolinealidad
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(to_drop, axis=1)
print(X.columns)

# Mostrar las primeras filas del conjunto de datos con la nueva columna 'ictus'
print(X.head())

Parte 3

In [ ]:

# Dividir el conjunto de datos en entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar modelo de regresión logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Hacer predicciones
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcular precisión
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

# Crear DataFrame con los resultados
results = pd.DataFrame({
    'Conjunto de Datos': ['Entrenamiento', 'Prueba'],
    'Precisión': [accuracy_train, accuracy_test]
})

# Mostrar los resultados
print(results)

NameError: name 'train_test_split' is not defined

Etapa 4

In [ ]:
#--------------------------Etapa 4: Evaluacion de modelos------------------------------
# Métricas de rendimiento para los datos de entrenamiento
y_pred_train = model.predict(X_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
f1_train = f1_score(y_train, y_pred_train)
balanced_accuracy_train = balanced_accuracy_score(y_train, y_pred_train)

# Tasa de verdaderos positivos (Recall) para los datos de entrenamiento
tp_train = ((y_train == 1) & (y_pred_train == 1)).sum()
fn_train = ((y_train == 1) & (y_pred_train == 0)).sum()
tpr_train = tp_train / (tp_train + fn_train)

# Tasa de verdaderos negativos (Especificidad) para los datos de entrenamiento
tn_train = ((y_train == 0) & (y_pred_train == 0)).sum()
fp_train = ((y_train == 0) & (y_pred_train == 1)).sum()
tnr_train = tn_train / (tn_train + fp_train)

# Métricas de rendimiento para los datos de prueba
y_pred_test = model.predict(X_test)
precision_test = precision_score(y_test, y_pred_test, zero_division=0)
recall_test = recall_score(y_test, y_pred_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)
balanced_accuracy_test = balanced_accuracy_score(y_test, y_pred_test)

# Matriz de confusión para los datos de prueba
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

print("\nMatriz de confusión para los datos de prueba:")
print(conf_matrix_test)

# Análisis de si el modelo está sobreajustado
if recall_train > recall_test:
    print("\nEl modelo está sobreajustado (overfitting) ya que el recall en los datos de entrenamiento es mayor que en los datos de prueba.")
else:
    print("\nEl modelo no está sobreajustado (overfitting).")

# Tasa de verdaderos positivos (Recall) para los datos de prueba
tp_test = ((y_test == 1) & (y_pred_test == 1)).sum()
fn_test = ((y_test == 1) & (y_pred_test == 0)).sum()
tpr_test = tp_test / (tp_test + fn_test)

# Tasa de verdaderos negativos (Especificidad) para los datos de prueba
tn_test = ((y_test == 0) & (y_pred_test == 0)).sum()
fp_test = ((y_test == 0) & (y_pred_test == 1)).sum()
tnr_test = tn_test / (tn_test + fp_test)

print("Métricas de rendimiento para los datos de entrenamiento:")
print(f"Precisión: {precision_train}")
print(f"Recall (Tasa de verdaderos positivos): {recall_train}")
print(f"Especificidad (Tasa de verdaderos negativos): {tnr_train}")
print(f"Precisión equilibrada: {balanced_accuracy_train}")
print(f"Puntuación F1: {f1_train}")

print("\nMétricas de rendimiento para los datos de prueba:")
print(f"Precisión: {precision_test}")
print(f"Recall (Tasa de verdaderos positivos): {recall_test}")
print(f"Especificidad (Tasa de verdaderos negativos): {tnr_test}")
print(f"Precisión equilibrada: {balanced_accuracy_test}")
print(f"Puntuación F1: {f1_test}")

# Calcular probabilidades predichas
y_prob_test = model.predict_proba(X_test)[:, 1]

# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob_test)

# Encontrar el umbral óptimo
optimal_threshold_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_threshold_idx]

# Convertir probabilidades predichas en clases binarias con el umbral optimo
y_pred_test_optimal = (y_prob_test >= optimal_threshold).astype(int)

# Calcular métricas de rendimiento con el umbral óptimo
precision_test_optimal = precision_score(y_test, y_pred_test_optimal)
recall_test_optimal = recall_score(y_test, y_pred_test_optimal)
accuracy_test_optimal = accuracy_score(y_test, y_pred_test_optimal)
f1_test_optimal = f1_score(y_test, y_pred_test_optimal)
balanced_accuracy_test_optimal = balanced_accuracy_score(y_test, y_pred_test_optimal)

# Comparar los resultados con el umbral por defecto
print("\nMétricas de rendimiento para los datos de prueba con umbral optimo:")
print(f"Precision: {precision_test_optimal}")
print(f"Recall (Tasa de verdaderos positivos): {recall_test_optimal}")
print(f"Precision equilibrada: {balanced_accuracy_test_optimal}")
print(f"Puntuacion F1: {f1_test_optimal}")

# Comparar con los resultados anteriores
print("\nComparacion con umbral por defecto:")
print(f"Precision: {precision_test}")
print(f"Recall (Tasa de verdaderos positivos): {recall_test}")
print(f"Precision equilibrada: {balanced_accuracy_test}")
print(f"Puntuacion F1: {f1_test}")